<a href="https://colab.research.google.com/github/mohitsaini65/Fraud-News-Detection-System/blob/main/Fraud_News_Detection_System_LSTM_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/Colab Notebooks/Fraud-News-Detection-System/archive (1).zip"
extract_path = "/content/drive/MyDrive/Colab Notebooks/Fraud-News-Detection-System/data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Unzipped successfully")

Unzipped successfully


In [ ]:
os.listdir("/content/drive/MyDrive/Colab Notebooks/Fraud-News-Detection-System/data")

['News _dataset']

In [ ]:
os.listdir("/content/drive/MyDrive/Colab Notebooks/Fraud-News-Detection-System/data/News _dataset")

['Fake.csv', 'True.csv']

In [ ]:
import pandas as pd

path = "/content/drive/MyDrive/Colab Notebooks/Fraud-News-Detection-System/data/News _dataset/"

fake = pd.read_csv(path + "Fake.csv")
true = pd.read_csv(path + "True.csv")

print(fake.shape, true.shape)


(23481, 4) (21417, 4)


In [ ]:
fake['label'] = 0   # Fake news
true['label'] = 1   # Real news

In [ ]:
import pandas as pd

df = pd.concat([fake, true], axis=0)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

df.head()

,title,text,subject,date,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",0
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",1
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",1
3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",0
4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",1


In [ ]:
df = df[['text', 'label']]
df.dropna(inplace=True)

df.head(), df.shape

(                                                text  label
 0  21st Century Wire says Ben Stein, reputable pr...      0
 1  WASHINGTON (Reuters) - U.S. President Donald T...      1
 2  (Reuters) - Puerto Rico Governor Ricardo Rosse...      1
 3  On Monday, Donald Trump once again embarrassed...      0
 4  GLASGOW, Scotland (Reuters) - Most U.S. presid...      1,
 (44898, 2))

In [ ]:
X = df['text']
y = df['label']

print(X.shape, y.shape)

(44898,) (44898,)


In [ ]:
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

corpus = []

for text in X:
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    words = text.split()
    words = [w for w in words if w not in stop_words]
    corpus.append(' '.join(words))



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from tensorflow.keras.preprocessing.text import one_hot

voc_size = 5000
onehot_repr = [one_hot(words, voc_size) for words in corpus]

onehot_repr[:2]

[[838,
  387,
  896,
  1044,
  4978,
  4460,
  681,
  4883,
  1119,
  1527,
  1732,
  4823,
  2032,
  429,
  3587,
  2127,
  2171,
  2006,
  3542,
  476,
  4254,
  2049,
  3765,
  4533,
  1217,
  4324,
  4429,
  189,
  1127,
  4900,
  4833,
  2542,
  4448,
  1063,
  3898,
  4008,
  4460,
  4036,
  1349,
  2626,
  2908,
  4394,
  3849,
  1636,
  327,
  2498,
  1063,
  3049,
  3221,
  4460,
  4051,
  509,
  503,
  4484,
  4504,
  3156,
  954,
  4504,
  1072,
  2944,
  1152,
  290,
  3765,
  4608,
  4614,
  882,
  1517,
  2303,
  2473,
  4967,
  598,
  4608,
  4394,
  4559,
  3838,
  4077,
  4560,
  2032,
  4462,
  4096,
  4512,
  4234,
  4900,
  3709,
  124,
  1063,
  3898,
  1667,
  4448,
  70,
  838,
  387,
  896,
  4448,
  3920,
  2547,
  1283,
  2913,
  904,
  896,
  3587],
 [3849,
  500,
  3290,
  2542,
  4000,
  4448,
  4888,
  4715,
  3891,
  3579,
  2527,
  344,
  2112,
  1210,
  3613,
  1008,
  3061,
  3138,
  4292,
  3267,
  3539,
  4504,
  3471,
  4041,
  2743,
  2112,
  4661,

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

sent_length = 20
embedded_docs = pad_sequences(
    onehot_repr,
    padding='pre',
    maxlen=sent_length
)

embedded_docs.shape

(44898, 20)

In [ ]:
import numpy as np

X_final = np.array(embedded_docs)
y_final = np.array(y)

X_final.shape, y_final.shape

((44898, 20), (44898,))

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_final,
    y_final,
    test_size=0.33,
    random_state=42
)

X_train.shape, X_test.shape

((30081, 20), (14817, 20))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

embedding_vector_features = 40

model_lstm = Sequential()
model_lstm.add(Embedding(voc_size, embedding_vector_features))
model_lstm.add(LSTM(100))
model_lstm.add(Dropout(0.3))
model_lstm.add(Dense(1, activation='sigmoid'))

model_lstm.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model_lstm.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model_lstm.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=5,
    batch_size=32
)


Epoch 1/5
941/941 ━━━━━━━━━━━━━━━━━━━━ 29s 26ms/step - accuracy: 0.8816 - loss: 0.2629 - val_accuracy: 0.9530 - val_loss: 0.1290
Epoch 2/5
941/941 ━━━━━━━━━━━━━━━━━━━━ 41s 26ms/step - accuracy: 0.9698 - loss: 0.0894 - val_accuracy: 0.9543 - val_loss: 0.1270
Epoch 3/5
941/941 ━━━━━━━━━━━━━━━━━━━━ 34s 36ms/step - accuracy: 0.9838 - loss: 0.0544 - val_accuracy: 0.9551 - val_loss: 0.1421
Epoch 4/5
941/941 ━━━━━━━━━━━━━━━━━━━━ 32s 26ms/step - accuracy: 0.9904 - loss: 0.0332 - val_accuracy: 0.9534 - val_loss: 0.1528
Epoch 5/5
941/941 ━━━━━━━━━━━━━━━━━━━━ 23s 24ms/step - accuracy: 0.9933 - loss: 0.0232 - val_accuracy: 0.9566 - val_loss: 0.1788


In [ ]:
y_pred = (model_lstm.predict(X_test) > 0.5).astype(int)

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print(confusion_matrix(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


464/464 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step
[[7406  348]
 [ 295 6768]]
Accuracy: 0.9566039009246137
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      7754
           1       0.95      0.96      0.95      7063

    accuracy                           0.96     14817
   macro avg       0.96      0.96      0.96     14817
weighted avg       0.96      0.96      0.96     14817



In [ ]:
model_lstm.build(input_shape=(None, sent_length))
model_lstm.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 20, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100)            │        56,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 769,505 (2.94 MB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 513,004 (1.96 MB)